In [ ]:
from keras import models
from keras import layers
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1118, activation='sigmoid'))
model.summary()

In [ ]:
import numpy as np
from array import array

def gray_to_rgb(img):
    return  np.squeeze(np.stack((img,) * 3, -1))

def batch_channel_convert(imgs):
    converted=np.empty([86086,32, 32,3])
    for index,img in enumerate(imgs):
        processed=gray_to_rgb(img)
        converted[index]=processed
    return converted

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import keras




X=pd.read_pickle("/kaggle/input/zoom-32-32-1/datas.pickle")
y=pd.read_pickle("/kaggle/input/zoom-32-32-1/labels.pickle")

convertedX=batch_channel_convert(X)
le=LabelEncoder()

y_labels=le.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(convertedX, y_labels, test_size=.5, random_state=42)


train_datagen = ImageDataGenerator(    
        featurewise_center=True,  
        samplewise_center=True,  
        featurewise_std_normalization=True,  
        samplewise_std_normalization=True,  
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest',
        rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen.fit(X_train)
test_datagen.fit(X_test)


train_datagen = train_datagen.flow(
    x=X_train,
    y=y_train,
    batch_size=559)
test_datagen = test_datagen.flow(
    x=X_test,y=y_test,
    batch_size=559)

callbacks_list = [
    keras.callbacks.EarlyStopping(
    monitor='acc',
    patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
    filepath='my_model.h5',
    monitor='val_loss',
    save_best_only=True,
    )]

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4,decay=1e-6),
              metrics=['acc'])

history = model.fit_generator(
      train_datagen,
      steps_per_epoch=77,
      epochs=100,
      callbacks=callbacks_list,
      validation_data=test_datagen,
      validation_steps=77,
      workers=4,
      verbose=1)

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))


plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
axes=plt.gca()
axes.set_xlim([0,len(epochs)])
axes.set_ylim([min(acc),max(acc)])
plt.title('Training and validation accuracy')
plt.legend()
plt.axes()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
axes=plt.gca()
axes.set_xlim([0,len(epochs)])
axes.set_ylim([min(loss),max(loss)])
plt.legend()
plt.axes()
plt.show()